In [1]:
import os
import re
import time
import csv
import json
import random
import requests
import pandas as pd
import numpy as np

from datetime import date, timedelta, datetime

import selenium
from selenium import webdriver
from bs4 import BeautifulSoup

### Set up the chrome driver by passing some parameters

In [36]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = False
browser = webdriver.Chrome(executable_path = '/Users/houping/Dropbox/My Mac (rcbm7256-diii)/Documents/Houping/chromedriver',
                          options = chrome_options)

In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = True
browser = webdriver.Chrome(
executable_path = '/Users/houping/Dropbox/My Mac (rcbm7256-diii)/Documents/Houping/chromedriver',
    chrome_options = chrome_options)


/Users/houping/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


### test

In [9]:
url_head = 'https://www.glassdoor.com/member/home/index.htm'

In [10]:
browser.get(url_head)
time.sleep(random.uniform(1,5))

#### filling in sign_in forms by identifying the username and password cells

In [38]:
def sign_in():
    url = 'https://www.glassdoor.com/profile/login_input.htm'
    browser.get(url)
    
    with open('credentials.json') as f:
        d = json.loads(f.read())
        username = d['glassdoor'][0]['username']
        password = d['glassdoor'][0]['password']

    email_field = browser.find_element_by_name('username')
    email_field.send_keys(username)
    submit_btn = browser.find_element_by_xpath('//button[@type="submit"]')
    submit_btn.click()
    time.sleep(2)
    
    password_field = browser.find_element_by_name('password')
    password_field.send_keys(password)
    submit_btn = browser.find_element_by_xpath('//button[@type="submit"]')
    submit_btn.click()

    time.sleep(1)

In [39]:
sign_in()

#### Test on American airlines company website on Glassdoor

Example: american airlines

In [41]:
# url = 'https://www.glassdoor.com/Overview/Working-at-ADC-Telecommunications-EI_IE1075.11,33.htm'
url = 'https://www.glassdoor.com/Overview/Working-at-American-Airlines-EI_IE8.11,28.htm'
browser.get(url)
time.sleep(1)

In [42]:
# overview section
reviews = browser.find_element_by_id('EIOverviewContainer')
overview = reviews.find_elements_by_class_name('align-items-center')

In [50]:
def scrape_overview(browser):
    # overview section
    overview = {}
    reviews = browser.find_element_by_id('EIOverviewContainer')
    # overview = reviews.find_elements_by_class_name('align-items-center')
    
    #website
    website = reviews.find_element_by_xpath(".//a[@data-test='employer-website']").text
    overview['website'] = website
    # headquarters
    headquarters =reviews.find_element_by_xpath(".//div[@data-test='employer-headquarters']").text
    overview['headquarters'] = headquarters
    # size
    size =reviews.find_element_by_xpath(".//div[@data-test='employer-size']").text
    overview['size'] = size
    # funded
    funded = reviews.find_element_by_xpath(".//div[@data-test='employer-founded']").text
    overview['funded'] = funded
    # type
    ctype = reviews.find_element_by_xpath(".//div[@data-test='employer-type']").text
    overview['ctype'] = ctype
    # industry
    industry = reviews.find_element_by_xpath(".//a[@data-test='employer-industry']").text
    overview['industry'] = industry
    # revenue
    revenue = reviews.find_element_by_xpath(".//div[@data-test='employer-revenue']").text
    overview['revenue'] = revenue
    # description
    try:
        reviews.find_element_by_xpath(".//span[@data-test='employerDescription']/button").click()
    except Exception:
        pass
    description = reviews.find_element_by_xpath(".//span[@data-test='employerDescription']").text
    overview['description'] = description
    # affiliated companies
    # employerHierarchies = reviews.find_element_by_xpath("//div[@data-test='employerHierarchies']")
    try:
        employerHierarchiesUrl = reviews.find_element_by_xpath("//div[@data-test='employerHierarchies']/a").get_attribute('href')
        browser.get(employerHierarchiesUrl)
        affiliatedCompanies = browser.find_elements_by_xpath("//a[@data-test='companyInfositeLink']")
        affiliatedAll = {}
        for i in range(len(affiliatedCompanies)):
            a = affiliatedCompanies[i]
            affiliated = {}
            affiliated['url'] = a.get_attribute('href')
            affiliated['companyName'] = a.find_element_by_xpath(".//h3[@data-test='employerName']").text
            affiliated['companyRating'] = a.find_element_by_xpath(".//p[@data-test='companyRating']").text
            affiliated['companyLocation'] = a.find_element_by_xpath(".//p[@data-test='companyLocation']").text
            affiliated['reviewsCount'] = a.find_element_by_xpath(".//p[@data-test='reviewsCount']").text
            affiliated['jobsCount'] = a.find_element_by_xpath(".//p[@data-test='jobsCount']").text
            affiliated['salariesCount'] = a.find_element_by_xpath(".//p[@data-test='salariesCount']").text
            
            affiliatedAll[i] = affiliated
        overview['affiliated'] = affiliatedAll
    except Exception:
        pass
    
    
    return overview

In [51]:
overview = scrape_overview(browser)

In [52]:
overview

{'website': 'jobs.aa.com',
 'headquarters': 'Fort Worth, TX',
 'size': '10000+ Employees',
 'funded': '1926',
 'ctype': 'Company - Public (AAL)',
 'industry': 'Airlines, Airports & Air Transportation',
 'revenue': '$10+ billion (USD)',
 'description': 'Embark on an adventure with a commitment to service, excellence and humanity. Our team is what powers our airline. We are proudly dedicated to our purpose of caring for people on life’s journey, including connecting our customers to the people and places they love or providing our team members development and leadership opportunities to acquire new skills and explore their potential.\n\nWith more than 1,300 aircraft in our mainline and regional fleets combined and an extensive route network that touches six continents, we’ve got a place for you to start — or continue — your career. Whether you’re stationed at one of our hundreds of airports, working out of our state-of-the-art headquarters in Fort Worth, Texas, or serving customers over 

### Extract urls for other sections

In [86]:
urlElements = browser.find_element_by_xpath("//div[@id='SmarterNavContainer']")

In [101]:
overviewUrl = urlElements.find_elements_by_tag_name('a')[0].get_attribute('href')
reviewsUrl = urlElements.find_elements_by_tag_name('a')[1].get_attribute('href')
jobsUrl = urlElements.find_elements_by_tag_name('a')[2].get_attribute('href')
salariesUrl = urlElements.find_elements_by_tag_name('a')[3].get_attribute('href')
interviewsUrl = urlElements.find_elements_by_tag_name('a')[4].get_attribute('href')
benefitsUrl = urlElements.find_elements_by_tag_name('a')[5].get_attribute('href')
photosUrl = urlElements.find_elements_by_tag_name('a')[6].get_attribute('href')
faqUrl = urlElements.find_elements_by_tag_name('a')[7].get_attribute('href')
locationUrl = urlElements.find_elements_by_tag_name('a')[8].get_attribute('href')